*Автор: Татьяна Рогович*

# Анализ данных в Python

## Введение в ML. Задача регрессии

Этот блокнот составлен на основе большого блокнота, который доступен по этой [ссылке](https://www.kaggle.com/dansbecker/your-first-machine-learning-model). В нем присутствуют дополнительные комментарии и другие примеры работы алгоритма. Если хотите как следует разобраться с деревьями на базовом уровне - обязательно читайте.

### Загружаем наши данные

Нашим датасетом будет информация о домах, которая включает в себя множество параметров, например, количество и размер этажей дома, год постройки, общая площадь, цена и многое другое (остальные столбцы, можете изучить самостоятельно).

In [1]:
import pandas as pd

#Загружаем данные
home_data = pd.read_csv('https://raw.githubusercontent.com/rogovich/Data/master/data/house_data.csv')

### Выбор целевой переменной

В нашем случае, все более-менее очевидно. Это цена на дом. Обычно, цель предсказания помечают y.

In [5]:
y = home_data.SalePrice

### Выбор признаков (Features)

Столбцы, которые есть в нашей модели и которые в последствии будут использованы для предсказания, называются признаками (features). В нашем случае, эти колонки будут определять стоимость дома. Иногда используются все колонки, кроме той на которую делается предсказание. А вдругих случаях, лучше выбрать только часть из них. Как определиться? Для этого проводят разведывательный анализ или сравнивают качество работы моделей на разных выборках признаков.

Мы будет использовать не все столбцы, чтобы алгоритм все считал быстрее. Добавим в модель только те столбцы, которые сильнее всего коррелировали с ценой, когда мы исследовали данные.

In [3]:
feature_columns = ['OverallQual', 'GrLivArea', 'GarageArea', 'TotalBsmtSF', 'FullBath', 'YearBuilt', 'YearRemodAdd']

Обычно, такие данные обозначаются X:

In [6]:
X = home_data[feature_columns]

Чтобы обучить модель, нам надо разбить нашу выборку на тестовую и тренировочную.

In [11]:
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.3)

Сколько элементов попало в подвыборки?

In [12]:
print(X_tr.shape, X_te.shape)

(1022, 7) (438, 7)


### Строим модель

Мы будем использовать модуль scikit-learn для создания модели. Scikit-learn одна из самых популярных библиотек для моделирования данных, хранящихся в датафреймах. 

Для построения модели нужно выполнить следующие шаги: <br>

**Define**: Какого типа будет модель? Линейная регрессия? Какой-то другой тип?<br>
**Fit**: запускаем модель на тренировочных данных (обучаем ее), чтобы алгоритм нашел в них некие закономерности и зависимости.<br>
**Predict**: Предсказать результат.<br>
**Evaluate**: Определить насколько точным оказалось предсказывание, оценить качество модели<br>


### Обучение линейной модели

Мы готовы к тому, чтобы построить модель машинного обучения. Обучим линейную модель на наших данных. Для этого импортируем LinearRegression из модуля sklearn:

In [13]:
from sklearn.linear_model import LinearRegression

Обучение модели в sklearn всегда состоит из двух шагов - создания модели и вызова функции fit:

In [14]:
model = LinearRegression()
model.fit(X_tr, y_tr)

LinearRegression()

После обучения "внутри" модели появились найденные веса:

In [15]:
model.coef_

array([ 18815.1959795 ,     63.8017906 ,     39.11048129,     39.61911436,
       -14400.93466253,    296.75472154,    338.55837944])

### Предсказания линейной модели

Выполним предсказания на двух выборках, обучающей и тестовой, и сохраним их в переменные preds_tr и preds_te. Для этого воспользуемся функцией predict:

In [16]:
preds_tr = model.predict(X_tr)
preds_te = model.predict(X_te)

Посмотрим на предсказания на первых 10 объектах:

In [17]:
preds_te[:10]

array([258547.81244547, 171649.15541205, 158246.97921184,  63275.3480804 ,
       111886.62128499, 221910.53114721, 164243.54011189,  81183.98234525,
        95463.86185533,  43930.06262375])

А вот целевые значения из выборки:

In [18]:
y_te[:10].values

array([268000, 185000, 143000,  80000, 129900, 226700, 153000, 111000,
        93000,  84000], dtype=int64)

### Оценивание качества

Сравнивать предсказания и правильные ответы вручную удобно только на маленькой выборке данных. Для оценивания качества работы модели лучше измерить метрику качества. Для этого в sklearn есть подмодуль metrics. 

Воспульзуемся метрикой MEA (Mean Absolute Error) - Cредняя Aбсолютная Ошибка. Ее можно представить в таком виде: ошибка = реальная цена − предсказанная цена. Например, если цена дома 150 000, мы предсказали цены в 100 000, то ошибка будет 50 000. Для ошибок в отрицательную сторону берем модуль. Это одна из простейших метрик.

Импортируем функцию mean_absolute_error.

In [19]:
from sklearn.metrics import mean_absolute_error 

Вычисляем ошибку на обучающей выборке:

In [20]:
mean_absolute_error(y_tr, preds_tr)

23317.199872831723

Вычисляем ошибку на тестовой выборке:

In [21]:
mean_absolute_error(y_te, preds_te)

25047.649235655757

### Анализ модели

Посмотрим на веса модели. Запишем их в таблицу со столбцами "название признака" и "вес признака" и отсортируем по значениям весов:

In [26]:
weights_data = {"веса":model.coef_, "признаки": X.columns}
weights = pd.DataFrame(weights_data)
weights.sort_values("веса")

,веса,признаки
4,-14400.934663,FullBath
2,39.110481,GarageArea
3,39.619114,TotalBsmtSF
1,63.801791,GrLivArea
5,296.754722,YearBuilt
6,338.558379,YearRemodAdd
0,18815.195980,OverallQual


Логично ли распределились вклады признаков?

### Задания для самостоятельной работы

Перед выполнением заданий обязательно выполните все ячейки выше. Для этого мжно нажать на эту ячейку, далее в меню Cell выбрать Run all above.

При выполнении заданий используйте примеры кода, данные выше, а также [__таблицу с подсказками__](https://github.com/nadiinchi/intro_sklearn/blob/master/Cheatsheet.pdf).

Мы обучили линейную модель на данных недвижимости Бостона. Теперь давайте рассмотрим еще две модели - метод k ближайших соседей (kNN) и нейронную сеть.

#### Задача 1. Обучение kNN

Обучите метод k ближайших соседей на данных. Метод уже импортирован в следующей ячейке.

In [27]:
from sklearn.neighbors import KNeighborsRegressor

In [28]:
model = KNeighborsRegressor()
model.fit(X_tr, y_tr)

KNeighborsRegressor()

#### Задача 2. Предсказания kNN

Выполните предсказания с помощью метода k ближайших соседей на обучающей и тестовой выборках.

In [29]:
preds_tr = model.predict(X_tr)
preds_te = model.predict(X_te)

#### Задача 3. Качество kNN

Выведите значение ошибки метода k ближайших соседей на обучающей и тестовой выборках.

In [30]:
print("Качество на обучении:", mean_absolute_error(y_tr, preds_tr))
print("Качество на контроле:", mean_absolute_error(y_te, preds_te))

Качество на обучении: 21598.762818003914
Качество на контроле: 25831.00182648402


#### Задача 4. Улучшаем kNN

У метода k ближайших соседей есть важный гиперпараметр - число соседей k. В sklearn он обозначен n_neighbors и задается следующим образом:

In [31]:
model = KNeighborsRegressor(n_neighbors=5)

Попробуйте использовать n_neighbors, равное 1, 3, 10, 100. Для каждого значения повторите шаги предыдущих трех задач: обучите модель, выполните предсказания, выведите ошибку на обучающей и тестовой выборке. Выберите n_neighbors с наименьшей ошибкой на тестовой выборке.

Усложенная версия: используйте цикл по четырем значениям n_neighbors.

In [34]:
model = KNeighborsRegressor(n_neighbors=5)

In [35]:
for k in [1, 3, 10, 100]:
    print("k =", k)
    model = KNeighborsRegressor(n_neighbors=k)
    model.fit(X_tr, y_tr)
    preds_tr = model.predict(X_tr)
    preds_te = model.predict(X_te)
    print("Качество на обучении:", mean_absolute_error(y_tr, preds_tr))
    print("Качество на контроле:", mean_absolute_error(y_te, preds_te))

k = 1
Качество на обучении: 67.78962818003914
Качество на контроле: 31662.388127853883
k = 3
Качество на обучении: 18760.45792563601
Качество на контроле: 25992.99619482496
k = 10
Качество на обучении: 24396.350391389435
Качество на контроле: 25353.666894977167
k = 100
Качество на обучении: 29961.59170254403
Качество на контроле: 28306.910547945205


# Решающее дерево и случайный лес

In [36]:
from sklearn.tree import DecisionTreeRegressor

Инициализируем модель и обучим ее на наших данных.

In [48]:
model = DecisionTreeRegressor()
model.fit(X_tr, y_tr)

preds_tr = model.predict(X_tr)
preds_te = model.predict(X_te)

print("Качество на обучении:", mean_absolute_error(y_tr, preds_tr))
print("Качество на контроле:", mean_absolute_error(y_te, preds_te))

Качество на обучении: 63.51696020874104
Качество на контроле: 28695.65068493151


### Эксперементируем с моделями

Можно сказать что мы столкнулись с проблемой переобучения (overfitting), когда модель предсказывает обучающие данные практически идеально, а тестовые данные и другие данные предсказывате плохо.

Существует еще и недообучение (underfitting), когда модель не может найти параметры по которым можно хорошо разделить данные, и она плохо предсказывает даже на обучающих данных.

Так как мы используем DecisionTreeRegressor, т.е. дерево, в случае overfitting дерево обычно очень глубокое, а при underfitting получаем деревья небольшие. Поэтому мы может попробовать регулировать глубину дерева. В DecisionTreeRegressor есть параметер max_leaf_nodes, который мы можем контролировать. Чем больше листьев мы разрешим модели делать, тем дальше мы уйдем от underfiting, но будем приближаться к overfitting, т.е. в идеальном случае нам нужно подобрать примерно среднюю глубину дерева для модели.

Создадим функцию, которая будет строить для нас деревеья с различным параметром max_leaf_nodes. А замерять модели будем снова через MAE.

In [49]:
def get_mae(max_leaf_nodes, X_tr, X_te, y_tr, y_te):
    # инициализируем модель с параметром max_leaf_node, который мы передали в функцию
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes)
    # обучаем модель на обучающих данных
    model.fit(X_tr, y_tr)
    # предсказываем на тестовых данных
    preds_val = model.predict(X_te)
    # считаем MAE
    mae = mean_absolute_error(y_te, preds_val)
    # возвращем значение ошибки
    return(mae)

В функцию мы соответственно передаем количество листов, которые мы хотим и обучающие и тестовые выборки. Сделаем небольшой цикл по различный значением глубины от 5 до 5000 и выведем результаты.

In [50]:
# сравниваем различне значения MAE для различной глубины деревьев
for max_leaf_nodes in [5, 10, 50, 100, 500, 1000, 5000]:
    my_mae = get_mae(max_leaf_nodes, X_tr, X_te, y_tr, y_te)
    print("Максимальная глубина дерева: %d  \t\t MEA:  %d" %(max_leaf_nodes, my_mae))

Максимальная глубина дерева: 5  		 MEA:  32632
Максимальная глубина дерева: 10  		 MEA:  28266
Максимальная глубина дерева: 50  		 MEA:  26492
Максимальная глубина дерева: 100  		 MEA:  27006
Максимальная глубина дерева: 500  		 MEA:  28830
Максимальная глубина дерева: 1000  		 MEA:  28108
Максимальная глубина дерева: 5000  		 MEA:  29173


Даже минимальная ошибка достаточно большая и искажает предсказания для новых данных. С другой стороны, если мы посмотрим описательные статистики для нашей цены, мы увидим, что по сравнению со медианой и интерквартильным размахом, наша ошибка не настолько ужасна. Поэтому величина ошибки предсказания достаточно хитрая штука для интерпретации.

In [19]:
train_y.describe()

count      1095.000000
mean     181712.286758
std       77955.082565
min       34900.000000
25%      130000.000000
50%      165000.000000
75%      215000.000000
max      745000.000000
Name: SalePrice, dtype: float64

### Random Forest

Дерево решений оставляет с серьезным выбором. Глубокое дерево с overfitting с несколькими домами в каждом листе или мелкое дерево с несколькими листами, которое не может определить параметры и разделить данные? И все это с приличной ошибкой.

Некоторые другие модели деревьев смогли решить эту проблему и уменьшить при этом итоговую ошибку. Попробуем взять Random Forest. Он использует множество деревьев и делает предсказание путем усреднения прогнозов каждого дерева.

[Подробнее про случайный лес тут.](https://dyakonov.org/2016/11/14/%D1%81%D0%BB%D1%83%D1%87%D0%B0%D0%B9%D0%BD%D1%8B%D0%B9-%D0%BB%D0%B5%D1%81-random-forest/)

Попробуем использовать Random Forest на наших данных. Импортируем его из модуля sklearn

In [43]:
from sklearn.ensemble import RandomForestRegressor

Строим теперь модель с тем же параметрами и данными как в DecissionTreeRegressor

In [51]:
model = RandomForestRegressor()
model.fit(X_tr, y_tr)

preds_tr = model.predict(X_tr)
preds_te = model.predict(X_te)

print("Качество на обучении:", mean_absolute_error(y_tr, preds_tr))
print("Качество на контроле:", mean_absolute_error(y_te, preds_te))

Качество на обучении: 7522.297649566675
Качество на контроле: 20141.70128082192


Ошибка уменьшилась, но все равно составляет больше 20 тысяч долларов. Можно продолжить дальше эксперементировать с моделью, изменяя параметры или увеличивая обучающую выборку. 

Давайте изменим параметр n_estimators, который отвечает за количество деревьев в модели ( не путать с количеством листьев в DecisionTree)

In [53]:
model = RandomForestRegressor(n_estimators=100)
model.fit(X_tr, y_tr)

preds_tr = model.predict(X_tr)
preds_te = model.predict(X_te)

print("Качество на обучении:", mean_absolute_error(y_tr, preds_tr))
print("Качество на контроле:", mean_absolute_error(y_te, preds_te))

Качество на обучении: 7601.838391668996
Качество на контроле: 19986.405668623614


Уже 19 тысяч! Много это или мало? Можно ли остановиться? 

Конечно, мы всегда стараемся минимизировать нашу ошибку, но это и не всегда возможно. Возможно, здесь лучше справится другая модель. Так же мы взяли очень мало признаков, при чем не основываясь на какой-то особой теории - возможно, преобразование наших признаков или же их замена, улучшили бы качество.

С остальными параметрами можно ознакомиться в [документации](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) функции

Дополнительным приятным бонусом использования моделей дерева и случайного леса является то, что мы можем использовать их определения важности признаков. Внутри себя дерево ранжирует признаки на основании того, насколько разбиение по ним снижает неоднородность данных. Так мы можем использовать лес для отбора наиболее важных признаков.

In [54]:
list(zip(feature_columns, forest_model.feature_importances_))

[('OverallQual', 0.6178126527948792),
 ('GrLivArea', 0.16075425274558722),
 ('GarageArea', 0.04613257208294265),
 ('TotalBsmtSF', 0.09255547156502697),
 ('FullBath', 0.015902805350439336),
 ('YearBuilt', 0.03647553637673043),
 ('YearRemodAdd', 0.030366709084394285)]

In [55]:
print("Features sorted by their score:")
print(sorted(zip(map(lambda x: round(x, 4), forest_model.feature_importances_), feature_columns), 
             reverse=True))

Features sorted by their score:
[(0.6178, 'OverallQual'), (0.1608, 'GrLivArea'), (0.0926, 'TotalBsmtSF'), (0.0461, 'GarageArea'), (0.0365, 'YearBuilt'), (0.0304, 'YearRemodAdd'), (0.0159, 'FullBath')]


Давайте сравним с матрицей корреляции. Если бы мы ориентировались только на корреляции, мы считали два первых признака почти одинаково важным. По важностям, определенными решающим деревом видно, что 'OverallQual' с большим отрывом опережает площадь.

In [56]:
home_data[feature_columns + ['SalePrice']].corr()['SalePrice']

OverallQual     0.790982
GrLivArea       0.708624
GarageArea      0.623431
TotalBsmtSF     0.613581
FullBath        0.560664
YearBuilt       0.522897
YearRemodAdd    0.507101
SalePrice       1.000000
Name: SalePrice, dtype: float64

На следующем семинаре мы вернемся к "Титанику" и попробуем, наконец, предсказать, кто же выжил во время крушения.